# Measurement Slice:

## Your Slice Settings/Locations

In [ ]:
import os 

# SSH keys to access your slice nodes
ssh_key_file_priv=os.environ['HOME']+"/.ssh/fabric_slice/id_rsa"
ssh_key_file_pub=os.environ['HOME']+"/.ssh/fabric_slice/id_rsa.pub"

# Ansible keys used for ansible installs on slice nodes from Meas_Node
# !These files will be written/overwritten when run!
ansible_key_file_priv = "demod_ansible"
ansible_key_file_pub = "demod_ansible.pub"

prom_hosts_file = "demod_hosts.ini"
elk_hosts_file = "demod_elk_hosts"
prom_vars_file = "demod_prom_vars.yml"

#? hosts.in and hostselk.ini? set filenames?

# Slice name & Site
slice_name='DemoD'
#MAX, TACC, MICH, mASS, UTAH, NCSA, RENC, UKY, LBNL
site='MAX'
image_name='default_ubuntu_20'

# For building SSH commands
# Bastion Host IPs
bastion_public_addr = 'bastion-1.fabric-testbed.net'
bastion_private_ipv4_addr = '192.168.11.226'
bastion_private_ipv6_addr = '2600:2701:5000:a902::c'
username = "ubuntu"

# Your Bastion Host Username and Key
bastion_username = "charles"  
bastion_private_key_file = os.environ['HOME'] + "/.ssh/extra_keys/unbridled_linux"  
vm_private_key_file=ssh_key_file_priv


In [ ]:
import os
from fabrictestbed.slice_manager import SliceManager, Status
import json
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType, ComponentModelType, ServiceType

# # Your Slice Settings
# ssh_key_file_priv=os.environ['HOME']+"/.ssh/id_rsa"
# ssh_key_file_pub=os.environ['HOME']+"/.ssh/id_rsa.pub"

# # Slice name & Site
# slice_name='MDCExample'
# site='TACC'
# image_name='default_ubuntu_20'

# # FOr building SSH commands
# # Bastion Host IPs
# bastion_public_addr = 'bastion-1.fabric-testbed.net'
# bastion_private_ipv4_addr = '192.168.11.226'
# bastion_private_ipv6_addr = '2600:2701:5000:a902::c'
# username = "ubuntu"

# # Your Bastion Host Username and Key
# bastion_username = "tristan_jordan_0041926686"  
# bastion_private_key_file ="/home/fabric/work/fabric_key"  #"/Documents/Fabric/keys/fabric_portal"
# vm_private_key_file = "/home/fabric/.ssh/id_rsa"



ssh_key = None
with open (ssh_key_file_pub, "r") as myfile:
    ssh_key=myfile.read()
    ssh_key=ssh_key.strip()
    
credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"CM Host: {credmgr_host} Orchestrator Host: {orchestrator_host}")
print(os.environ['FABRIC_CREDMGR_HOST'])

slice_manager = SliceManager(oc_host=orchestrator_host, cm_host=credmgr_host, project_name='MF', scope='all')

# Initialize the slice manager
slice_manager.initialize()

# Create topology
myExperiment = ExperimentTopology()

# Add nod
Node1 = myExperiment.add_node(name="Node1", site=site)
Node2 = myExperiment.add_node(name="Node2", site=site)
Node3 = myExperiment.add_node(name="Node3", site=site)
#MeasNGINX = myExperiment.add_node(name="Meas_NGINX", site=site)
MeasNode = myExperiment.add_node(name="Meas_Node", site=site)

# Set capacities

cap = Capacities()
cap.set_fields(core=2, ram=16, disk=100)

meas_cap = Capacities()
meas_cap.set_fields(core=4, ram=16, disk=100)

# Set Properties
Node1.set_properties(capacities=cap, image_type='qcow2', image_ref=image_name)
Node2.set_properties(capacities=cap, image_type='qcow2', image_ref=image_name)
Node3.set_properties(capacities=cap, image_type='qcow2', image_ref=image_name)
#MeasNGINX.set_properties(capacities=cap, image_type='qcow2', image_ref=image_name)
MeasNode.set_properties(capacities=meas_cap, image_type='qcow2', image_ref=image_name)

Node1.add_component(model_type=ComponentModelType.SharedNIC_ConnectX_6, name='MDCnic1')
#Node1.add_component(model_type=ComponentModelType.SharedNIC_ConnectX_6, name='MDCnic2')
Node2.add_component(model_type=ComponentModelType.SharedNIC_ConnectX_6, name='MDCnic3')
#Node2.add_component(model_type=ComponentModelType.SharedNIC_ConnectX_6, name='MDCnic4')
Node3.add_component(model_type=ComponentModelType.SharedNIC_ConnectX_6, name='MDCnic5')
#Node3.add_component(model_type=ComponentModelType.SharedNIC_ConnectX_6, name='MDCnic6')
#MeasNGINX.add_component(model_type=ComponentModelType.SharedNIC_ConnectX_6, name='MDCnic7')
#MeasNGINX.add_component(model_type=ComponentModelType.SharedNIC_ConnectX_6, name='MDCnic8')
MeasNode.add_component(model_type=ComponentModelType.SharedNIC_ConnectX_6, name='MDCnic9')
#MeasNode.add_component(model_type=ComponentModelType.SharedNIC_ConnectX_6, name='MDCnic10')


print(myExperiment.interface_list)
# L2Bridge Service
myExperiment.add_network_service(name='ExperimentBridge', nstype=ServiceType.L2Bridge, interfaces=myExperiment.interface_list)

# L2Bridge Service
#myExperiment.add_network_service(name="MeasurementBridge", nstype=ServiceType.L2Bridge, interfaces=[myExperiment.interface_list[1],myExperiment.interface_list[3],myExperiment.interface_list[5],myExperiment.interface_list[7],myExperiment.interface_list[9]])

# Generate Slice Graph
slice_graph = myExperiment.serialize()

# Request slice from Orchestrator
status, reservations = slice_manager.create(slice_name=slice_name, slice_graph=slice_graph, ssh_key=ssh_key)

print("Response Status {}".format(status))
if status == Status.OK:
    print("Reservations created {}".format(reservations))
else:
    print(f"Failure: {reservations}")


# List Slices

In [ ]:
import os
from fabrictestbed.slice_manager import SliceManager, Status, SliceState
import json

return_status, slices = slice_manager.slices(excludes=[SliceState.Dead])
slice = list(filter(lambda x: x.slice_name == slice_name, slices))[0]
print(slice)
print(return_status)

In [ ]:
import time
def wait_for_slice(slice,timeout=180,interval=10,progress=False):
    timeout_start = time.time()

    if progress: print("Waiting for slice .", end = '')
    while time.time() < timeout_start + timeout:
        return_status, slices = slice_manager.slices(excludes=[SliceState.Dead,SliceState.Closing])

        if return_status == Status.OK:
            slice = list(filter(lambda x: x.slice_name == slice_name, slices))[0]
            if slice.slice_state == "StableOK":
                if progress: print(" Slice state: {}".format(slice.slice_state))
                return slice
            if slice.slice_state == "Closing" or slice.slice_state == "Dead":
                if progress: print(" Slice state: {}".format(slice.slice_state))
                return slice    
        else:
            print(f"Failure: {slices}")
        
        if progress: print(".", end = '')
        time.sleep(interval)
    
    if time.time() >= timeout_start + timeout:
        if progress: print(" Timeout exceeded ({} sec). Slice: {} ({})".format(timeout,slice.slice_name,slice.slice_state))
        return slice    

slice = list(filter(lambda x: x.slice_name == slice_name, slices))[0]
slice = wait_for_slice(slice, progress=True)
print()

print("Slice Name : {}".format(slice.slice_name))
print("ID         : {}".format(slice.slice_id))
print("State      : {}".format(slice.slice_state))
print("Lease End  : {}".format(slice.lease_end))

# Run after slice is "StableOK"

In [ ]:

return_status, experiment_topology = slice_manager.get_slice_topology(slice_object=slice)
print(return_status)
for node_name, node in experiment_topology.nodes.items():
    
    print("Node:")
    print("   Name              : {}".format(node.name))
    print("   Cores             : {}".format(node.get_property(pname='capacity_allocations').core))
    print("   RAM               : {}".format(node.get_property(pname='capacity_allocations').ram))
    print("   Disk              : {}".format(node.get_property(pname='capacity_allocations').disk))
    print("   Image             : {}".format(node.image_ref))
    print("   Image Type        : {}".format(node.image_type))
    print("   Host              : {}".format(node.get_property(pname='label_allocations').instance_parent))
    print("   Site              : {}".format(node.site))
    print("   Management IP     : {}".format(node.management_ip))
    print("   Reservation ID    : {}".format(node.get_property(pname='reservation_info').reservation_id))
    print("   Reservation State : {}".format(node.get_property(pname='reservation_info').reservation_state))
    print("   Components        : {}".format(node.components))
    print("   Interfaces        : {}".format(node.interfaces))
    print(node)    

# Extend Slice Life

In [ ]:
import datetime
import os
from fabrictestbed.slice_manager import SliceManager, Status, SliceState
import json
#Slice renew to end in 1 day
end_date = (datetime.datetime.now() + datetime.timedelta(days=7)).strftime("%Y-%m-%d %H:%M:%S")

status, result = slice_manager.renew(slice_object=slice,
                                     new_lease_end_time = end_date)
print("Stat {}".format(status))
print("Response Status {}".format(result))
return_status, slices = slice_manager.slices(excludes=[SliceState.Dead,SliceState.Closing])

if return_status == Status.OK:
    slice = list(filter(lambda x: x.slice_name == slice_name, slices))[0]

    print("Slice Name : {}".format(slice.slice_name))
    print("ID         : {}".format(slice.slice_id))
    print("State      : {}".format(slice.slice_state))
    print("Lease End  : {}".format(slice.lease_end))
else:
    print(f"Failure: {slices}")

# SSH builder

In [ ]:
# Get the ssh commands for accessing slice nodes
return_status, experiment_topology = slice_manager.get_slice_topology(slice_object=slice)
print(return_status)
# For user
print("{0} user accounts________________________________________________________________________".format(username))
for node_name, node in experiment_topology.nodes.items():
    print(node_name)
    print('ssh -i {} -i {} -J {}@{} {}@{}'.format(bastion_private_key_file,
                                               vm_private_key_file,
                                               bastion_username,
                                               bastion_public_addr,
                                               username,
                                               node.management_ip))
# For ansible
print("\nAnsible user accounts________________________________________________________________________")
for node_name, node in experiment_topology.nodes.items():
    print(node_name)
    print('ssh -i {} -i {} -J {}@{} {}@{}'.format(bastion_private_key_file,
                                               ansible_key_file_priv,
                                               bastion_username,
                                               bastion_public_addr,
                                               "ansible",
                                               node.management_ip))
    

for node_name, node in experiment_topology.nodes.items():
    if node_name == "Meas_Node":
        print('To login to grafana use:')
        print('ssh -L 10010:localhost:443 -i {} -i {} -J {}@{} {}@{}'.format(bastion_private_key_file,
                                               vm_private_key_file,
                                               bastion_username,
                                               bastion_public_addr,
                                               username,
                                               node.management_ip))

        print("Then browse to https://localhost:10010/grafana")
        print("then type in user and password set in prom vars found elsewhere in this notebook")
        print("to change graphs then login with door to bottom left as admin with password found elsewhere in this notebook")
      

# Setup IP

In [ ]:
import paramiko
hosts = []
baseip = 0
ips = ["192.168.10.52","192.168.10.53","192.168.10.54","192.168.10.55","192.168.10.56"]
return_status, experiment_topology = slice_manager.get_slice_topology(slice_object=slice)
for node_name, node in experiment_topology.nodes.items():
    script= 'sudo ip addr add '+ips[baseip]+'/24 dev ens7 \n sudo ip link set dev ens7 up \n '
    #ip addr show\nip link show\n
    host_entry = "{0} ansible_host={1} hostname={1} ansible_ssh_user={2} node_exporter_listen_ip={1}".format(node_name,  ips[baseip],"ansible")
    hosts.append(host_entry)

    
    baseip += 1
    try:
        management_ip = str(node.management_ip)
        print("Node {0} IP {1}".format(node_name, management_ip))
        

        key = paramiko.RSAKey.from_private_key_file(vm_private_key_file)

        bastion=paramiko.SSHClient()
        bastion.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        bastion.connect(bastion_public_addr, username=bastion_username, key_filename=bastion_private_key_file)

    
        bastion_transport = bastion.get_transport()
        src_addr = (bastion_private_ipv6_addr, 22)

        dest_addr = (management_ip, 22)
        bastion_channel = bastion_transport.open_channel("direct-tcpip", dest_addr, src_addr)


        client = paramiko.SSHClient()
        client.load_system_host_keys()
        client.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())
        client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        client.connect(management_ip,username=username,pkey = key, sock=bastion_channel)

        stdin, stdout, stderr = client.exec_command(script )
        print ('')
        print (str(stdout.read(),'utf-8').replace('\\n','\n'))
        print (str(stderr.read(),'utf-8').replace('\\n','\n'))

        client.close()
    except Exception as e:
        print(str(e))





# Generate Ansible key pair

In [ ]:
from cryptography.hazmat.primitives import serialization as crypto_serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.backends import default_backend as crypto_default_backend
from os import chmod

key = rsa.generate_private_key(
    backend=crypto_default_backend(),
    public_exponent=65537,
    key_size=2048
)

private_key = key.private_bytes(
    crypto_serialization.Encoding.PEM,
    crypto_serialization.PrivateFormat.TraditionalOpenSSL,
    crypto_serialization.NoEncryption()
)

public_key = key.public_key().public_bytes(
    crypto_serialization.Encoding.OpenSSH,
    crypto_serialization.PublicFormat.OpenSSH
)

# Decode to printable strings
private_key_str = private_key.decode('utf-8')
public_key_str = public_key.decode('utf-8')

# Save public key & change mode
public_key_file = open(ansible_key_file_pub, 'w');
public_key_file.write(public_key_str);
public_key_file.write('\n');
public_key_file.close()
chmod(ansible_key_file_pub, 0o644);

# Save private key & change mode
private_key_file = open(ansible_key_file_priv, 'w');
private_key_file.write(private_key_str);
private_key_file.close()
chmod(ansible_key_file_priv, 0o600);

# Print key (for debug)
print(public_key_str)
print(private_key_str)

# Host file creation

In [ ]:
        
print("Creating Hosts File")
        
#Prometheus
hosts_txt = ""
experiment_nodes = "[Experiment_Nodes]\n"
for host in hosts:
    if host.startswith("Meas_Node"):
        hosts_txt += "[Meas_Node]\n"
        hosts_txt += host + '\n\n'
        
    elif host.startswith("Meas_NGINX"):
        hosts_txt += "[Meas_NGINX]\n"
        hosts_txt += host + '\n\n'
        
    else: # It is an experiemnters node
        experiment_nodes += host + '\n'
hosts_txt += experiment_nodes
print(hosts_txt)
with open(prom_hosts_file, 'w') as f:
    f.write(hosts_txt)

#Elk
hosts_txt = ""
experiment_nodes = "[workers]\n"
for host in hosts:
    if host.startswith("Meas_Node"):
        hosts_txt += "[elk]\n"
        hosts_txt += host + '\n\n'
        
    elif host.startswith("Meas_NGINX"):
        hosts_txt += "[Meas_NGINX]\n"
        hosts_txt += host + '\n\n'
        
    else: # It is an experiemnters node
        experiment_nodes += host + '\n'
hosts_txt += experiment_nodes
print(hosts_txt)
with open(elk_hosts_file, 'w') as f:
    f.write(hosts_txt)
    
    
prom_vars = '''---
fabric_prometheus_ht_user: <username_here>
fabric_prometheus_ht_password: <user_password_here>

node_exporter_username: node_exporter
node_exporter_password: <password_here>

snmp_community_string: not-in-use

grafana_admin_password: <grafana_admin_password_here>

'''    
with open(prom_vars_file, 'w') as f:
    f.write(prom_vars)

# Setup Ansible Users

In [ ]:
#SCP public on all nodes, private on measurement (Ansible user)
return_status, experiment_topology = slice_manager.get_slice_topology(slice_object=slice)
import paramiko

def scp_file(ip, port, username, keyfile, srcfile, destfile):
    print("SCPing file: {0}".format(srcfile))
    management_ip = str(ip)
    key = paramiko.RSAKey.from_private_key_file(keyfile)

    bastion=paramiko.SSHClient()
    bastion.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    bastion.connect(bastion_public_addr, username=bastion_username, key_filename=bastion_private_key_file)


    bastion_transport = bastion.get_transport()
    src_addr = (bastion_private_ipv6_addr, 22)

    dest_addr = (management_ip, 22)
    bastion_channel = bastion_transport.open_channel("direct-tcpip", dest_addr, src_addr)


    client = paramiko.SSHClient()
    client.load_system_host_keys()
    client.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(management_ip,username=username,pkey = key, sock=bastion_channel)
    sftp = client.open_sftp()
    sftp.put(srcfile, destfile)
    sftp.close()
    client.close()
    
def run_commands(ip, port, username, keyfile, command):
    print("Running command: {0}".format(command))
    management_ip = str(ip)

    key = paramiko.RSAKey.from_private_key_file(keyfile)

    bastion=paramiko.SSHClient()
    bastion.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    bastion.connect(bastion_public_addr, username=bastion_username, key_filename=bastion_private_key_file)


    bastion_transport = bastion.get_transport()
    src_addr = (bastion_private_ipv6_addr, 22)

    dest_addr = (management_ip, 22)
    bastion_channel = bastion_transport.open_channel("direct-tcpip", dest_addr, src_addr)


    client = paramiko.SSHClient()
    client.load_system_host_keys()
    client.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(management_ip,username=username,pkey = key, sock=bastion_channel)

    stdin, stdout, stderr = client.exec_command(command )
    print ('')
    print (str(stdout.read(),'utf-8').replace('\\n','\n'))
    print (str(stderr.read(),'utf-8').replace('\\n','\n'))

    client.close()
        
for node_name, node in experiment_topology.nodes.items():
    try:
        #Skip all nodes except host
        #if node_name != "Meas_Node":
        #    continue
        print(node_name)
        curip = str(node.management_ip)
        print("________________Setting Up ansible user on {0} at {1}_____________________".format(node_name, curip))
        

        # SETUP ANSIBLE USER
        # Create Ansible User
        run_commands(curip, "22", username, vm_private_key_file, "sudo useradd -G root -m ansible")

        # Create Ansible User .ssh dir
        run_commands(curip, "22", username, vm_private_key_file, " sudo mkdir /home/ansible/.ssh; sudo chmod 700 /home/ansible/.ssh; sudo chown -R ansible:ansible /home/ansible/.ssh")

        # Make ansible as sudo user without password
        run_commands(curip, "22", username, vm_private_key_file, "echo \"ansible ALL=(ALL:ALL) NOPASSWD: ALL\" | sudo tee /etc/sudoers.d/ansible")
        
        
        # SETUP KEYS
        # Copy pub keys to all nodes
        # Can't scp to ansible dir yet, so we have to scp, then move to ansible user
        scp_file(curip, "22", username, vm_private_key_file, ansible_key_file_pub, "ansible.pub")

        # Move to ansible dir & change ownership
        run_commands(curip, "22", username, vm_private_key_file, "sudo mv ansible.pub /home/ansible/.ssh/ansible.pub; sudo chown ansible:ansible /home/ansible/.ssh/ansible.pub;")

        # Cat to authorized keys
        run_commands(curip, "22", username, vm_private_key_file, "sudo cat /home/ansible/.ssh/ansible.pub | sudo tee -a /home/ansible/.ssh/authorized_keys")
        # Ensure authorized keys has correct permissions and owner
        run_commands(curip, "22", username, vm_private_key_file, "sudo chmod 644 /home/ansible/.ssh/authorized_keys; sudo chown ansible:ansible /home/ansible/.ssh/authorized_keys")
   
        
        if(node.name=="Meas_Node"):
            scp_file(curip, "22", username, vm_private_key_file, ansible_key_file_priv, "ansible")
            run_commands(curip, "22", username, vm_private_key_file, "sudo mv ansible /home/ansible/.ssh/ansible; sudo chown ansible:ansible /home/ansible/.ssh/ansible; sudo chmod 600 /home/ansible/.ssh/ansible")

        print( "_______Ansible User Setup Completed_______")
    except Exception as e:
        print("Error in transfer: " + str(e))


# Setup Ansible to Install Prometheus & ELK

In [ ]:
for node_name, node in experiment_topology.nodes.items():
    #print( node_name )
    try:
        print("________{0}".format(node_name))
        #run_commands(curip, 22, "ansible", ansible_key_file_priv,  "sudo apt update; sudo apt install -y python3-pip" ) 

                    
        if (node_name == "Meas_Node"):
            curip = str(node.management_ip)
            print("Setting up {0} at {1}".format(node_name, curip))
            
            #run_commands(curip, 22, "ansible", ansible_key_file_priv, "pwd")
            
            # Clone repo & run bootstrap shell script found in the repo
            run_commands(curip, 22, "ansible", ansible_key_file_priv, "git clone https://github.com/fabric-testbed/MeasurementFramework.git mf_git;  cd mf_git; git pull; git checkout add_fabric_experiment_role; experiment_bootstrap/install_ansible.sh")
           
            #scp_file(curip, "22", "ansible", "ansible.test.key", "hosts.ini", "mf_git/ansible/hosts/hosts.ini")
            
            # Copy over hosts file for bootstrap ansible to install docker
            scp_file(curip, "22", "ansible", ansible_key_file_priv, prom_hosts_file, "mf_git/ansible/fabric_experiment_instramentize/hosts")        
            # Install Docker 
            run_commands(curip, 22, "ansible", ansible_key_file_priv, "cd /home/ansible/mf_git/ansible/fabric_experiment_instramentize; /home/ansible/.local/bin/ansible-playbook playbook_fabric_experiment_install_docker.yml")

            # Install Prometheus System
            # copy over vars file
            scp_file(curip, "22", "ansible", ansible_key_file_priv, prom_vars_file, "mf_git/ansible/fabric_experiment_instramentize/prom_vars.yml")
            run_commands(curip, 22, "ansible", ansible_key_file_priv, 'cd mf_git/ansible/fabric_experiment_instramentize; /home/ansible/.local/bin/ansible-playbook --key-file "~/.ssh/ansible" -i hosts playbook_fabric_experiment_install_prometheus.yml --extra-vars "@prom_vars.yml" --diff')
            
            # Copy elk hosts file
            #scp_file(curip, "22", "ansible", ansible_key_file_priv, "hostselk.ini", "mf_git/elk/hosts")
            # Install Elk
            #run_commands(curip, 22, "ansible", ansible_key_file_priv, "cd mf_git/elk; sudo sed -i 's/Meas_Node/" + curip + "/' site.yml; /home/ansible/.local/bin/ansible-playbook site.yml;")
            
            print("_____________________________Monitoring Install Complete______________________________________")
    except Exception as e:
        print("Error in transfer: " + str(e))

# Delete Slice

In [ ]:
import os
from fabrictestbed.slice_manager import SliceManager, Status, SliceState
import json
status, slices = slice_manager.slices(excludes=[SliceState.Closing, SliceState.Dead])

if status == Status.OK:
    slice_object=list(filter(lambda s: s.slice_name == slice_name, slices))[0]
else:
    print(f"Failure: {slices}")
status, result = slice_manager.delete(slice_object=slice_object)

print("Response Status {}".format(status))
print("Response received {}".format(result))